In [9]:
import requests
import pandas as pd
import psycopg2 
from psycopg2.extras import execute_values

In [10]:
url = "https://pokeapi.co/api/v2/"
url_dynamic = "pokemon/pikachu"
headers = {"Accept-Encoding": "gzip, deflate"}

response = requests.get(url + url_dynamic, headers=headers)
data = response.json()


In [11]:
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="caromponce_coderhouse"
with open("D:\Coder House\Data Engineering Flex\Proyectos\Entregable1\pwd.txt",'r') as f:
    pwd= f.read()

try:
    conn = psycopg2.connect(
        host=url,
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Connected to Redshift successfully!")
    
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)


Connected to Redshift successfully!


In [12]:
def load_in_redshift(conn, data, table_name):
    cols= ["id", "name", "base_experience", "height", "weight", "types"]
    sql_dtypes=["INT", "VARCHAR(50)", "INT", "INT", "INT", "VARCHAR(100)"]
    column_defs = [f"{name} {data_type}" for name, data_type in zip(cols, sql_dtypes)]
    table_schema = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {', '.join(column_defs)}
        );
        """
    # Crear la tabla
    cur = conn.cursor()
    cur.execute(table_schema)
    # # Definir el INSERT
    insert_sql = f"INSERT INTO {table_name} ({', '.join(cols)}) VALUES %s"
    # Execute the transaction to insert the data
    cur.execute("BEGIN")
    execute_values(cur, insert_sql, data)
    cur.execute("COMMIT")
    print('Proceso terminado')

In [13]:
list_types_names = [item["type"]["name"] for item in data["types"]]
concatenated_types_names = ', '.join(list_types_names)

data_to_send = [(data["id"], data["name"], data["base_experience"], data["height"], data["weight"], concatenated_types_names)]

In [14]:
load_in_redshift(conn, data_to_send, "pokemon")

Proceso terminado
